# Setting up an external computer

## Aim

We can use aiida to submit jobs to an external computer. In this notebook we will: configure a profile, setup a computer, configure the SSH transport layer and create a code.

## Setup

For this tutorial we will assume you have:

<ul>
        <li>Aiida setup and can run jobs locally</li>
        <li>A database setup:</li>
        <ul>
                <li>user: aiida_stfc</li> 
                <li>database: aiida_stfc_db</li>
        </ul>
        <li>Quantum Espresso setup on the remote computer</li>
</ul>

### Profile

Run the following cell to create a yaml file with the settings of the profile. Replacing [XXX] with appropriate settings

In [ ]:
%%writefile ~/profile.yaml
profile: scarf
email: [EMAIL]
first_name: [NAME]    
last_name: [LASTNAME]
institution: [INSTITUTION]
database_username: aiida_stfc
database_password: [PASSWORD]
database_name: aiida_stfc_db
repository_uri: file:///home/[USERNAME]/.aiida/repository/scarf

Setup the profile ( **NOTE**: For this example we are using `core.psql_dos` (Postgres), but you can use `core.sqlite_dos` (SQLite) by replacing `core.psql_dos` in command line params and changing the configuration file. Use `verdi profile setup core.sqlite_dos -h` for more details on how to setup config file ):

In [ ]:
! verdi profile setup core.sqlite_dos --config ~/profile.yaml --non-interactive --set-as-default

### Computer

Now we setup the computer with following settings. ( **NOTE**: This setup is to connect to SCARF, you should change the label, description, hostname and workdir respectively inline with your system setting )

In [ ]:
%%writefile ~/computer.yaml
label: scarf
description: https://www.scarf.rl.ac.uk/index.html
hostname: ui1.scarf.rl.ac.uk
transport: core.ssh
scheduler: core.slurm
shebang: '#!/bin/bash'
work_dir: /work4/scd/scarf1480/aiida
mpirun_command: srun -u -n {tot_num_mpiprocs}
mpiprocs_per_machine: 32
use_double_quotes: False
prepend_text: ''
append_text: ''

Run the following command

In [ ]:
! verdi computer setup -n --config ~/computer.yaml

Once the computer is created, you then have to configure the ssh transport to communicate with the computer. Replace username, key_filename and gss_host with your respective settings.

In [ ]:
%%writefile ~/ssh_transport.yaml
username: [USERNAME]
port: 22
look_for_keys: True
key_filename: [SSH_KEY_PATH]
timeout: 60
allow_agent: True
proxy_command: ''
proxy_jump: ''
compress: True
gss_auth: False
gss_kex: False
gss_deleg_creds: False
gss_host: ui1.scarf.rl.ac.uk
load_system_host_keys: True
key_policy: AutoAddPolicy
use_login_shell: True
safe_interval: 30.0

In [ ]:
! verdi computer configure core.ssh scarf -n --config ~/ssh_transport.yaml

Once the compuuter is setup, you can test the connection

In [ ]:
! verdi computer test scarf

### External Code

If you want to run a calculation that is only available on the external machine, you can set up a code which accesses the external library. `verdi code setup -h` goes into more detail about what each of options does

In [ ]:
%%writefile ~/qe_code.yaml
label: qe
description: Quantum ESPRESSO pw.x
default_calc_job_plugin: quantumespresso.pw
computer: scarf
filepath_executable: /work4/scd/scarf562/eb-amd/software/QuantumESPRESSO/7.2-foss-2023a/bin/pw.x
use_double_quotes: true
prepend_text: |+
  module load amd-modules
  module load QuantumESPRESSO/7.2-foss-2023a

append_text: ' '

In [ ]:
! verdi code create core.code.installed -n --config ~/qe_code.yaml